In [1]:
import torch
from torch import nn
from torch.nn import functional as F

### 自定义块

In [6]:
class MLP(nn.Module):
    # 用init方法声明网络的层次结构
    def __init__(self):
        # 调用父类Module的构造函数来执行必要的初始化
        # 这样在类实例化时也可以指定其他函数参数,例如模型参数params
        super().__init__()
        # 隐藏层
        self.hidden = nn.Linear(20, 256)
        # 输出层
        self.out = nn.Linear(256, 10)

    # 定义模型的正向传播,即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意:这里我们使用ReLU的函数版本,其在nn.functional模块中定义
        return self.out(F.relu(self.hidden(X)))


net = MLP()
X = torch.rand(2, 20)
net(X)

tensor([[-0.0824,  0.0753,  0.1267,  0.2812,  0.1587, -0.0436,  0.0294,  0.1297,
          0.1116, -0.0391],
        [-0.0793,  0.0860, -0.0221,  0.2465,  0.0553,  0.0068,  0.1423,  0.1995,
         -0.0367, -0.0254]], grad_fn=<AddmmBackward0>)

### 顺序块

In [7]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            # block是Module子类的一个实例,把它保存在Module类的成员变量_modules中(OrderedDict类型)
            self._modules[block] = block

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X


net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
X = torch.rand(2, 20)
net(X)

tensor([[-0.1698,  0.0152,  0.0542,  0.2061,  0.2325,  0.0515, -0.0873,  0.1390,
         -0.2947,  0.0201],
        [-0.0384, -0.0030,  0.0714,  0.1796,  0.1691,  0.0817, -0.0391,  0.1210,
         -0.1731, -0.0262]], grad_fn=<AddmmBackward0>)

### 作用

* 可以在进行许多自定义的操作，例如在正向传播函数中执行代码
* 有可能导致效率问题 (Python的全局解释器锁问题)